# Doppio clustering

## Import

In [ ]:
import numpy as np
import pandas as pd
import os
import h5py
import random
import sys
import corner
import matplotlib.pyplot as plt
import taurex.log
import importlib
import seaborn as sns
import posterior_utils
import ast
import numpy as np
import matplotlib.pyplot as plt
import warnings

# Importa dinamicamente il modulo helper
import helper
importlib.reload(helper) 
check_parameters_valid = helper.check_parameters_valid

# Aggiungi il percorso della directory contenente helper.py
sys.path.append(os.path.abspath('./'))

importlib.reload(posterior_utils)

from matplotlib.lines import Line2D
from sklearn.preprocessing import normalize
from sklearn.mixture import GaussianMixture
from sklearn import mixture
from helper import *
from preprocessing import *
from submit_format import to_competition_format
from posterior_utils import *
from spectral_metric import *
from FM_utils_final import *
from sklearn.preprocessing import StandardScaler
from matplotlib.lines import Line2D
from submit_format import get_unique_filename
from sklearn.exceptions import ConvergenceWarning

# plt.style.use('default')
#plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['red'])

taurex.log.disableLogging()
np.set_printoptions(suppress=True, linewidth=np.nan, threshold=sys.maxsize)

# Filtering dei Warning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message="more than 1 Rp value detected in the trace! Using first value")

## Setting variabili

In [ ]:
n_repeat = 5
random_state = 420
# random_state = random.randint(1,1000)
# print(f'Random State : {random_state}')

# Lettura dati dai file 
aux = np.load('aux.npy')
spec_matrix = np.load('spectra.npy')
noise = np.load('noise.npy')
labels = np.load('label.npy')
validTraces = np.load('validTraces.npy')
num_spectra = spec_matrix.shape[0]
labels_names = ['planet_radius','planet_temp','log_H2O','log_CO2','log_CO','log_CH4','log_NH3']

# Setting dei path
training_path = './Check_Dataset/TrainingData'
training_GT_path = os.path.join(training_path, 'Ground Truth Package')
trace_GT = h5py.File(os.path.join(training_GT_path, 'TraceData.hdf5'), "r")
validTraces = validTraces.astype(np.int64)

for X in trace_GT.keys():
    tr_GT = trace_GT[X]['tracedata'][()]
    weights_GT = trace_GT[X]['weights'][()]
    if np.isnan(tr_GT).sum() == 1:
        continue
    validTraces = np.append(validTraces, int(X[12:]))

vt = validTraces
test_ind = np.sort(vt - 1)
train_ind = np.setdiff1d(np.arange(num_spectra), test_ind)
plot_ind = random.sample(range(len(test_ind)), 10)
spectra_ind = random.sample(range(len(test_ind)), 10)

# Preprocessing dei dati spettrali

# Preprocessamento dei test spectra
test_spectra = spec_matrix[test_ind, :]
test_spectra = augment_data(test_spectra, noise[test_ind, :], repeat=1)
test_spectra = test_spectra.reshape(-1, spec_matrix.shape[1])

# Preprocessamento dei train spectra
train_spectra = spec_matrix[train_ind, :]
train_spectra = augment_data(train_spectra, noise[train_ind, :], repeat=n_repeat)
train_spectra = train_spectra.reshape(-1, spec_matrix.shape[1])

# Applichiamo StandardScaler per preservare meglio la distribuzione originale
scaler = StandardScaler()
train_spectra = scaler.fit_transform(train_spectra)
test_spectra = scaler.transform(test_spectra)

# Dati ausiliari e labels
train_aux = aux[train_ind, :]
train_aux = np.repeat(train_aux, repeats=n_repeat, axis=0)
test_aux = aux[test_ind, :]
train_labels = labels[train_ind, :]
train_labels = np.repeat(train_labels, repeats=n_repeat, axis=0)
test_labels = labels[test_ind, :]

# Applicazione del Clustering
- $K_1$ : numero di cluster di 1o livello
- $K_2$ : numero di 'sottocluster' (cluster di 2o livello)
- $GMM_i$ : output del Gaussian Mixture Model
- $Labels_i$ : labels ottenute dal GMM
> Cerchiamo, in ogni sottocluster (K2), i cluster contenenti un singolo spettro, che sono considerati *outlier*, cioe' anomali (composti da un solo spettro)
### Si puo' usare il BIC (o l'AIC) per ottenere **i valori ottimali** di $K_1$ e $K_2$

In [ ]:
# (19,10) sembra essere la migliore coppia per il clustering [Risultato BIC]
K1 = 19
K2 = 5
GMM_i = []
Labels_i = []
gmm = mixture.GaussianMixture(n_components=K1, random_state=random_state, max_iter=400).fit(train_aux)
labels_1 = gmm.predict(train_aux)
for i in range(K1):
    spectra_i = np.where(labels_1 == i)[0]
    print("Spettri nel cluster #", i, " -> ", len(spectra_i))
    # Secondo clustering (2° livello) sui dati spettrali appartenenti al cluster i
    tmp = mixture.GaussianMixture(n_components=K2, random_state=random_state).fit(train_spectra[spectra_i, :])
    labels_2 = tmp.predict(train_spectra[spectra_i, :])
    GMM_i.append(tmp)
    Labels_i.append(labels_2)
    for j in range(K2):
        spectra_j = np.where(labels_2 == j)[0]
        if len(spectra_j) == 1:
            print(f"\t [{i}].{j} sottocluster con singolo spettro [OUTLIER]")
            #spectra_i = np.delete(spectra_i,spectra_j) # Rimozione dell'outlier

## Setup per gli score

In [ ]:
posterior_scores = []
spectral_scores = []
bounds_matrix = default_prior_bounds()
beta = 0.8
q_list = np.linspace(0.01, 0.99, 10)
opacity_path = "./XSEC/"
CIA_path = "./HITRAN"
ariel_wlgrid, ariel_wlwidth, ariel_wngrid, ariel_wnwidth = ariel_resolution()
fm = initialise_forward_model(opacity_path, CIA_path)
RJUP = 69911000
MJUP = 1.898e27
RSOL = 696340000
Rs = aux[:, 2] / RSOL
Mp = aux[:, 4] / MJUP
n_samples = 1000

### Calcolo degli score

In [ ]:
spec_max = 20
n_spec = 0
spec_ind = random.sample(range(len(test_ind)),spec_max*2)
k1_vals = []
k2_vals = []
spec_maxes = []
clustering_vals = []
subclustering_vals = []
posterior_scores_k = []
spectral_scores_k = []
scores_k = []
spec_maxes = []
spec_inds = []

for X in range(len(test_ind)):
	idx1 = gmm.predict(test_aux[X, :].reshape(1, -1))[0]
	km = GMM_i[idx1]
	labels_2 = Labels_i[idx1]
	idx2 = km.predict(test_spectra[X, :].reshape(1, -1))[0]
	idx_1 = np.where(labels_1 == idx1)[0]
	idx_2 = np.where(labels_2 == idx2)[0]
	cluster_membs = idx_1[labels_2==idx2]
	lab = train_labels[cluster_membs,:]
	lab = lab.reshape(-1,lab.shape[-1])
	posterior = lab
	weights1 = np.ones((posterior.shape[0],1)) / posterior.shape[0]
	planet_index = test_ind[X]+1
	tr_GT = trace_GT[f'Planet_train{planet_index}']['tracedata'][()] # Corrected Planet ID format
	wh_GT = trace_GT[f'Planet_train{planet_index}']['weights'][()] # Corrected Planet ID format
	if np.isnan(tr_GT).sum() >= 1:
		print("nan trovato")
		exit()
	score = compute_posterior_loss(posterior, weights1, tr_GT, wh_GT,bounds_matrix)
	posterior_scores.append(score)
	if n_spec<spec_max and X in spec_ind:
		try:
			proxy_compute_spectrum = setup_dedicated_fm(fm, test_ind[X], Rs, Mp, ariel_wngrid, ariel_wnwidth )
			sscore = compute_spectral_loss(posterior, weights1, tr_GT, wh_GT, bounds_matrix, proxy_compute_spectrum, q_list)
			spectral_scores.append(sscore)
			print('*****Spectral: ',sscore)
			print('*****Posterior: ',score)
			spec_inds.append(X)
			n_spec+=1
		except RuntimeWarning:
			continue

# Calcolo degli score medi
- Posterior Score : 80%
- Spectral Score : 20%
- Final score = Score finale per la Leaderboard

In [ ]:
avg_posterior_score = np.mean(posterior_scores)
print(f'Posterior_Score: {avg_posterior_score}')
avg_spectral_score = np.mean(spectral_scores)
print(f'Spectral_Score: {avg_spectral_score}')
final_score = (1 - beta) * avg_spectral_score + beta * avg_posterior_score
print(f"final score: {final_score:.4f}")

k1_vals.append(K1)
k2_vals.append(K2)
# clustering_vals.append((K1,K2))
clustering_vals.append(K1)
subclustering_vals.append(K2)
cluster_ids = list(range(len(clustering_vals)))
posterior_scores_k.append(avg_posterior_score)
spectral_scores_k.append(avg_spectral_score)
spec_maxes.append(spec_max)
scores_k.append(final_score)
with open("results.txt", "a") as f:
    f.write("\n------------------------------\n")
    f.write("K1: {}\n".format(k1_vals))
    f.write("K2: {}\n".format(k2_vals))
    f.write("Posterior scores: {}\n".format(posterior_scores_k))
    f.write("Spectral scores: {}\n".format(spectral_scores_k))
    f.write("Final scores: {}\n".format(scores_k))
f.close()

## Plotting grafico Score/Clustering

In [ ]:
score_plot_ind = 0
# Leggi l'intero contenuto del file
with open("results.txt", "r") as f:
    content = f.read() 

# Suddividi in blocchi
blocks = [block for block in content.split('------------------------------') if block.strip()]

# Inizializza le liste
k_pairs = []  # Salva le coppie (K1,K2)
posterior_list = []
spectral_list = []
final_list = []

def safe_literal_eval(value):
    try:
        value = value.strip().lower()
        if value == "nan":
            return np.nan
        return ast.literal_eval(value)
    except (SyntaxError, ValueError):
        print(f"Errore nel parsing: {value}")
        return None

for block in blocks:
    k1, k2 = None, None
    posterior = spectral = final = None
    
    lines = [line.strip() for line in block.strip().splitlines() if line.strip()]
    for line in lines:
        if line.startswith("K1:"):
            k1 = safe_literal_eval(line.split(":", 1)[1].strip())
        elif line.startswith("K2:"):
            k2 = safe_literal_eval(line.split(":", 1)[1].strip())
        elif line.startswith("Posterior"):
            posterior = safe_literal_eval(line.split(":", 1)[1].strip())
        elif line.startswith("Spectral"):
            spectral = safe_literal_eval(line.split(":", 1)[1].strip())
        elif line.startswith("Final"):
            final = safe_literal_eval(line.split(":", 1)[1].strip())
    
    if all(v is not None for v in [k1, k2, posterior, spectral, final]):
        k_pairs.append(f"({k1},{k2})")
        posterior_list.append(np.mean(posterior) if isinstance(posterior, (list, tuple, np.ndarray)) else posterior)
        spectral_list.append(np.mean(spectral) if isinstance(spectral, (list, tuple, np.ndarray)) else spectral)
        final_list.append(np.mean(final) if isinstance(final, (list, tuple, np.ndarray)) else final)

# Crea il plot con coppie K1,K2
plt.figure(figsize=(12, 6))
x_pos = np.arange(len(k_pairs))  # Posizioni per le barre/ticks

# Line plot per i tre score
plt.plot(x_pos, posterior_list, marker='o', color='red', label='Posterior Score', linestyle='--')
plt.plot(x_pos, spectral_list, marker='s', color='black', label='Spectral Score', linestyle='-.')
plt.plot(x_pos, final_list, marker='D', color='blue', label='Final Score', linestyle=':')

# Formattazione assi
plt.xticks(x_pos, k_pairs, rotation=45, ha='right', fontsize=8)
plt.xlabel('(K1, K2)', fontweight='bold')
plt.ylabel('Scores', fontweight='bold')
plt.title('Confronto Scores per Configurazioni (K1,K2)', pad=20)
plt.legend()
plt.grid(True, alpha=0.3)

# Aggiungi valori sugli ultimi punti
for i, (p, s, f) in enumerate(zip(posterior_list, spectral_list, final_list)):
    plt.annotate(f'{p:.1f}', (x_pos[i], p), textcoords="offset points", xytext=(0,5), ha='center', color='red', fontsize=8)
    plt.annotate(f'{s:.1f}', (x_pos[i], s), textcoords="offset points", xytext=(0,5), ha='center', color='black', fontsize=8)
    plt.annotate(f'{f:.1f}', (x_pos[i], f), textcoords="offset points", xytext=(0,5), ha='center', color='blue', fontsize=8)

plt.tight_layout()
plot_fname = get_unique_filename('Grafico','.png','./Cluster_Scores/')
plt.savefig(plot_fname)
plt.show()
csv_fname = 'results.csv'
df_results = pd.DataFrame({
    'K1' : [K1],
    'K2' : [K2],
    'FinalScore' : [final_score]
})
if os.path.exists(csv_fname):
    df_results.to_csv(csv_fname,mode='a',header=False,index=False)
else:
    df_results.to_csv(csv_fname,mode='w',header=True,index=False)


# Grafico andamento clustering
### Fissato K1, il grafico indica l'andamento del final_score per il variare di K2

In [ ]:
df = pd.read_csv('results.csv')
# print(df.head())
df.columns = ['K1','K2','final_score']
df = df.sort_values(by=['K1','K2'])
k1_values = sorted(df['K1'].unique())
plt.figure(figsize=(8,6))
colormap = plt.cm.get_cmap('tab10', len(k1_values))
for i,k1 in enumerate(k1_values):
    subset = df[df['K1']==k1]
    plt.plot(
        subset['K2'],
        subset['final_score'],
        marker='o',
        label=f'K1={k1}',
        color=colormap(i)
    )
plt.xticks(np.arange(1,22,2))
plt.xlabel('K2')
plt.ylabel('Final Score')
plt.title('Andamento score al variare di K2, per K1')
plt.legend()
plt.grid(True)
y_min, y_max = plt.ylim()  # prendi i limiti correnti
margine = (y_max - y_min) * 0.1  # 10% di margine
plt.ylim(y_min - margine, y_max + margine)
fname = get_unique_filename('Grafico','.png',directory='./Grafici_Clustering')
plt.savefig(fname)
plt.show()

# Plotting delle distribuzioni
### Si utiliza 'corner' per il plot delle distribuzioni bayesiane a posteriori

In [ ]:
'''
for X in plot_ind:
	#dist1 = pairwise_distances(kmeans.cluster_centers_,aux[X,:].reshape(1,-1))
	#idx1 = np.argmin(dist1)
	idx1 = gmm.predict(test_aux[X,:].reshape(1,-1))[0]

	km = GMM_i[idx1]
	labels_2 = Labels_i[idx1]
	#dist2 = pairwise_distances(km.cluster_centers_,spec_matrix[X,:].reshape(1,-1))
	#idx2 = np.argmin(dist2)
	idx2 = km.predict(test_spectra[X,:].reshape(1,-1))[0]
	#score = km.score_samples(spec_matrix[X,:].reshape(1,-1))[0]

	idx_1 = np.where(labels_1 == idx1)[0]
	idx_2 = np.where(labels_2 == idx2)[0]

	lab = train_labels[idx_1[idx_2],:]

	mean_lab = np.mean(lab,axis=0)
	mean_GT = np.average(tr_GT,axis=0,weights=wh_GT)

	tr_GT = trace_GT[f'Planet_train{test_ind[X]+1}']['tracedata'][()]
	wh_GT = trace_GT[f'Planet_train{test_ind[X]+1}']['weights'][()]

	# TRACES
	figure = corner.corner(tr_GT,quiet=True) #,weights=wh_GT
	# Extract the axes
	axes = np.array(figure.axes).reshape((tr_GT.shape[1], tr_GT.shape[1]))
	
	legend_elements = [
        Line2D([0], [0], color='blue', lw=2, label='Distribuzione Reale (tr_GT)'),
        Line2D([0], [0], color='red', lw=2, label='Distribuzione Predetta (lab)'),
        Line2D([0], [0], color='green', lw=2, label='Valore Vero (test_labels)'),
        Line2D([0], [0], color='black', lw=2, label='Media Predetta (mean_lab)')
    ]
    
	# Loop over the diagonal
	for i in range(tr_GT.shape[1]):
		ax = axes[i, i]
		ax.sharex(axes[tr_GT.shape[1]-1,i])
		ax.axvline(test_labels[X,i], color="g",lw=2)
		ax.axvline(mean_lab[i], color="r",lw=1.5)
		ax.axvline(mean_GT[i],color="blue",lw=2)
	    #ax.axvspan(mean_lab[i] - std_lab[i], mean_lab[i] + std_lab[i], alpha=0.3, color='red')
		ax.relim()
		ax.autoscale()
		ax.set_title(labels_names[i])
	
	# Aggiungi la legenda all'angolo in alto a destra
	figure.legend(
        handles=legend_elements, 
        loc='upper right', 
        bbox_to_anchor=(0.92, 0.92),
        frameon=True,
        framealpha=0.9
    )

	figure.set_figheight(8.5)
	figure.set_figwidth(12)
	# PREDICTED
	corner.corner(lab,fig=figure,quiet=True, color='red')
	#SAVE
	# figure.savefig(f'./Plots/k1-{K1}_k2-{K2}_{spec_max}spec/Planet_'+str(test_ind[X]+1)+'.png')
	figure.savefig('./GMM_plots/Planet_'+str(test_ind[X]+1)+'.png')
plt.show()
'''